In [1]:
import numpy as np
import os
import tqdm
import matplotlib.pyplot as plt
import math
import cv2
import skimage
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data.dataset import Dataset
from torchvision import datasets, models, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import random
from scipy import misc
import time
import copy
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image

In [2]:
class MyRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return TF.rotate(x, angle)

rotation_transform = MyRotationTransform(angles=[-90, 0, 90, 180])
transformations = transforms.Compose([
        transforms.Resize((128,128)), 
        rotation_transform,
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [3]:
class CustomDatasetFrom(Dataset): # input np arrays
    def __init__(self,nparray,labarray,nblabels,transform=None):
        self.data = nparray
        self.label = labarray
        self.nlabels = nblabels
        self.transforms = transform
    def __getitem__(self,index):
        img_as_np = self.data[index]
        img_as_img = Image.fromarray(img_as_np)
        if self.transforms is not None:
            img_as_tensor = self.transforms(img_as_img)
        y=torch.tensor(np.array([[self.label[index]]]),dtype=torch.long)   
        return img_as_tensor, y
    def __len__(self):
        return self.data.shape[0]

In [4]:
def loadData(imgPATH='D:/DreamAI/cards'):
    dfs = pd.read_excel("cardLaberlling.xlsx")
    dfs = dfs.drop(np.arange(2500,5000,1))
    distinctNames = np.unique(np.array(dfs['card name']).astype(str))
    dataDIR = imgPATH
    allcardIDs = os.listdir(dataDIR)
    imgArr,labelArr = [],[]
    for j in tqdm.tqdm(allcardIDs):
        tmpID = int(j.replace('card.png',''))
        if tmpID <= 2500: 
            tmpDIR = os.path.join(dataDIR,j)
            tmpdata = Image.open(tmpDIR).convert('RGB')
            tmppat = np.array(dfs['card name'][dfs['card']==tmpID])[0]
            imgArr.append(np.array(tmpdata))
            try:
                labelArr.append(np.where(distinctNames==tmppat)[0][0])
            except:
                labelArr.append(np.where(distinctNames=='nan')[0][0])
            
    return np.array(imgArr),np.array(labelArr),distinctNames

In [5]:
imgArr,labelArr,classNames = loadData()

 41%|███████████████████████████████                                             | 2043/5000 [00:00<00:00, 3033.49it/s]C:\Users\44754\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  app.launch_new_instance()
100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 694.56it/s]


In [6]:
nb_labels = len(classNames)

In [7]:
custom_data = CustomDatasetFrom(imgArr,labelArr,nb_labels,transformations)
dataloaders = torch.utils.data.DataLoader(custom_data, batch_size=4,
                                    shuffle=True, num_workers=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
validation_split,test_split = .05,.05
# Creating data indices for training and validation splits:
dataset_size = len(custom_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
test_indices, val_indices = indices[:split], indices[split:split*2]
train_indices = indices[split*2:]
print(len(train_indices),len(test_indices),len(val_indices))

2250 125 125


In [9]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(custom_data, batch_size=4, 
                                          sampler=train_sampler, num_workers=0)
validation_loader = torch.utils.data.DataLoader(custom_data, batch_size=4,
                                          sampler=valid_sampler, num_workers=0)
test_loader = torch.utils.data.DataLoader(custom_data, batch_size=4,
                                          sampler=test_sampler, num_workers=0)

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataload = train_loader
                datasize = len(train_loader)
            else:
                model.eval()   # Set model to evaluate mode
                dataload = validation_loader
                datasize = len(validation_loader)

            running_loss = 0.0
            running_corrects = 0
            running_exp = 0
            # Iterate over data.
            for inputs, labels in tqdm.tqdm(dataload):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    target = labels.squeeze()
                    if not target.size():
                        target = labels.view(-1)
                    loss = criterion(outputs, target)                    

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == target.data)
                running_exp += torch.sum(preds == preds)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / running_exp.double()
            epoch_acc = running_corrects.double() / running_exp.double()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Define My custom model

$$n_{i+1}=\frac{n_i-f+2p}{s}+1$$

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,256,2,1,1)
        self.conv1_drop = nn.Dropout2d()
        self.conv2 = nn.Conv2d(256,128,2,1,1)
        self.conv2_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(128,64,2,1,1)
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(16*16*64, 512)
        self.fc2 = nn.Linear(512, nb_labels)
    def forward(self, x):
        x = self.conv1_drop(F.max_pool2d(F.relu(self.conv1(x)),2)) #128
        x = self.conv2_drop(F.max_pool2d(F.relu(self.conv2(x)),2)) #64
        x = self.conv3_drop(F.max_pool2d(F.relu(self.conv3(x)),2)) #32
        x = x.view(-1, 16*16*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return x

In [12]:
learning_rate,momentum = 0.01,0.5
model = Net(); model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [13]:
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [00:24<00:00, 22.58it/s]


train Loss: 3.7273 Acc: 0.1009


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 58.39it/s]


val Loss: 2.4986 Acc: 0.2640
Epoch 1/29
----------


...

val Loss: 0.2279 Acc: 0.9760
Epoch 29/29
----------


100%|████████████████████████████████████████████████████████████████████████████████| 563/563 [00:25<00:00, 22.44it/s]


train Loss: 0.3464 Acc: 0.9076


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 57.40it/s]


val Loss: 0.2378 Acc: 0.9760
Training complete in 13m 24s
Best val Acc: 0.976000


In [16]:
def runmodel(model,dataloaders,device):
    was_training = model.training
    model.eval()
    labs = []; total = 0; correct = 0
    with torch.no_grad():
        for i, (inputs,labels) in enumerate(dataloaders):
            inputs = inputs.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            try:
                total += len(labels.numpy().squeeze())
            except:
                total += 1
            for j in range(inputs.size()[0]):
                tmp = preds[j].cpu().numpy()
                try:
                    correct += (labels.numpy().squeeze()[j]==tmp)
                except:
                    correct += (labels.numpy().squeeze()==tmp)
                labs.append(tmp)
        print('Test Accuracy: {}'.format(correct/total*100))
    return np.array(labs)

In [21]:
labelsTest = runmodel(model_ft,test_loader,device)

Test Accuracy: 96.8


In [20]:
torch.save(model_ft, os.path.join(os.getcwd(),'cardtopscratch.pt'))

C:\Users\44754\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\44754\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\44754\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\44754\Anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [22]:
modelmemory = torch.load(os.path.join(os.getcwd(),'cardtopscratch.pt'))